In [ ]:
from subgrounds.subgrounds import Subgrounds
from subgrounds.pagination import ShallowStrategy
import pandas as pd
import requests as r
import sys
sys.path.append('../')
import defillama_utils as dfl
import optimism_pool_tvls as tvls
from datetime import timedelta
import time
import os

### Data Sources
- Messari Subgraphs: [https://thegraph.com/hosted-service/subgraph/messari/curve-finance-ethereum](https://thegraph.com/hosted-service/subgraph/messari/curve-finance-ethereum)

In [ ]:
chain_list = ['arbitrum','aurora','avalanche','ethereum',\
        'fantom','gnosis','harmony','moonbeam','optimism','polygon']
trailing_num_days = 180
start_time = pd.to_datetime("today") - timedelta(days = trailing_num_days)
start_time = int( time.mktime(start_time.timetuple()) )

# def get_messari_sg_pool_snapshots(slug, chains, min_ts, max_ts):
get_curve = tvls.get_messari_sg_pool_snapshots('curve-finance',chain_list,start_time)


### Export and Show

In [ ]:
if not os.path.exists('csv_outputs'):
    os.makedirs('csv_outputs')
get_curve.to_csv('csv_outputs/curve_snapshot_data.csv')
# display(get_curve)

In [ ]:
# group by the specified columns
grouped = get_curve.sort_values(by='timestamp', ascending=False)\
                  .groupby(['protocol_network', 'pool_id', 'pool_name', 'pool_symbol'])

# perform the desired aggregations
aggregated = grouped.agg({'totalValueLockedUSD': 'first', 'dailyVolumeUSD': 'mean', 'timestamp':'nunique'})

aggregated = aggregated.rename(columns={'totalValueLockedUSD':'latest_tvl_usd'
                                        ,'dailyVolumeUSD':'avg_daily_volume_usd'
                                        ,'timestamp':'days_of_trading'})

# reset the index
aggregated = aggregated.reset_index()

aggregated = aggregated[aggregated['pool_id'] != 0]

display(aggregated)
aggregated.to_csv('csv_outputs/curve_aggregated_data.csv')